In [ ]:
# cantidad de datos: laps y vehicles
vehicles = sorted(data['vehicle_id'].unique())

# datos por cada vehiculo
veh_rows = {}
for veh in vehicles: 
    veh_rows[veh] = len(data[data['vehicle_id'] == veh])

print('# Max laps: ', len(data['lap'].unique()), '->',data['lap'].unique())
print('# Max de datos: ', max(veh_rows.values()))
print('# Min de datos: ', min(veh_rows.values()))
print(veh_rows.items())

In [ ]:
def dis(row, df):
    row['dis'] = round(hvs(row['latitude_longitude'], df.loc[row.name+1, 'latitude_longitude'], unit=Unit.METERS), 2)
    row['travel_time'] = time_travel(row, df)
    row['end_index'] = row.name+1
    return row

tb_fveh = pd.DataFrame(columns=['vehicle', 'lap',  'total_p', 'ida_p', 'sum_dis'])
list_stop = []
list_dis = []
list_time = []
dict_df = {}

for veh in vehicles[0:1]:
    df_veh = data[data['vehicle_id'] == veh].reset_index(drop=True).copy()
    laps = sorted(df_veh['lap'].unique()[1:-1])

    # print(veh, laps)
    # laps = [3]
    for lap in laps:
        df_veh_lap = idaPoints(df_veh[df_veh['lap'] == lap].reset_index(drop=True))

        if df_veh_lap.shape[0] > 99:
            # calcular la distancia entre puntos
            df_veh_lap =  df_veh_lap.head(df_veh_lap.shape[0]-1).apply(lambda row: dis(row, df_veh_lap), axis=1)
            list_dis += list(df_veh_lap['dis'])
            list_time += list(df_veh_lap['travel_time'])
            dict_df[veh+str(lap)] = df_veh_lap
            
            # verificar puntos por stop
            df_veh_lap_stop = df_veh_lap.apply(lambda row: SearchPoints(row, df_stops), axis=1)
            df_veh_lap_stop = df_veh_lap_stop[df_veh_lap_stop['stop'].notna()].reset_index()
            list_stop += list(df_veh_lap_stop['stop'])

            # registrar
            tb_fveh.loc[tb_fveh.shape[0]] = [veh, lap, df_veh.shape[0], df_veh_lap.shape[0], df_veh_lap['dis'].sum()]
        else: 
            print(veh, lap, df_veh_lap.shape[0])

# print( len(list_dis))
tmp = [ (k, v) for k,v in dict(Counter(list_stop)).items()]
tb_fstop = pd.DataFrame(tmp, columns =['id_stop', 'total_p'])
tb_fstop.sort_values(by='total_p', inplace=True, ascending=False, ignore_index=True)
# print(tb_fstop.to_string())


In [ ]:
import matplotlib.pyplot as plt

tb_fstop.sort_values(by='id_stop', inplace=True, ascending=True, ignore_index=True, key=np.int64)

# creating the bar plot
plt.figure(figsize = (30, 10))
plt.bar(list(tb_fstop['id_stop']), list(tb_fstop['total_p']), color ='maroon',width = 0.4)
 
plt.xlabel("Stops")
plt.ylabel("Total points")
plt.title("Numero de puntos por stop con r = 60m")
plt.show()
plt.close()

id_stop = list(map(str, df_stops_ida['id']))
set(id_stop) - set(tb_fstop['id_stop'])

In [ ]:
df = dict_df['SJ012']
df.shape

In [ ]:
plt.figure(figsize=(30, 10))
plt.plot(df.index, df['dis'])
plt.show()
plt.close()
print(df['dis'].mean())

plt.figure(figsize=(30, 10))
plt.plot(df.index, df['travel_time'])
plt.show()
plt.close()
print(df['travel_time'].mean())

plt.figure(figsize=(30, 10))
plt.plot(df.index, df['velocity'])
plt.show()
plt.close()
print(df['velocity'].mean())

In [ ]:
samples_with_nan = pd.read_csv('samples_with_nan.csv', index_col=None)
#samples_with_nan.nlargest(50,columns='time_travel')
#samples_with_nan[samples_with_nan['time_travel']>=960]['id_sample'].unique #960 segundos equivale a 10 minutos
samples_with_nan[samples_with_nan['time_travel']<12]['id_sample'].unique().shape #960 segundos equivale a 10 minutos

In [ ]:
# completar los nan con el promedio
for gg in list_samples:
    i , l_nan = 0, []
    while i < gg.shape[0]:
        if math.isnan(gg.loc[i, 'time_travel']):
            l_nan.append(i)
            gg.loc[i, 'time_travel'] = means[i]
        else:
            if len(l_nan) != 0:
                if 0 not in l_nan:
                    init = datetime.strptime(str(gg.loc[l_nan[0]-1, 'date_time_end']), "%Y-%m-%d %H:%M:%S")  
                    end = datetime.strptime(str(gg.loc[l_nan[-1]+1, 'date_time_init']), "%Y-%m-%d %H:%M:%S")   
                    dis_time = (end - init).total_seconds()
                    me = [means[x] for x in l_nan]
                    tmp = list(map(lambda x: round(dis_time*(x/sum(me)), 0), me))
                    tiempos = dict(zip(l_nan, tmp))
                    for j in l_nan:
                        gg.loc[j, 'time_travel'] = tiempos[j]

                    for k in l_nan:
                        gg.loc[k, 'date_time_init'] = gg.loc[k-1, 'date_time_end']
                        gg.loc[k, 'date_time_end'] = datetime.strptime(str(gg.loc[k, 'date_time_init']), "%Y-%m-%d %H:%M:%S") + timedelta(seconds=gg.loc[k, 'time_travel'])
                else:
                    for k in l_nan[::-1]:
                        gg.loc[k, 'date_time_end'] = gg.loc[k+1, 'date_time_init']
                        gg.loc[k, 'date_time_init'] = datetime.strptime(str(gg.loc[k, 'date_time_end']), "%Y-%m-%d %H:%M:%S") - timedelta(seconds=gg.loc[k, 'time_travel'])
                l_nan = []
        if i == 82 and len(l_nan) != 0:
            for k in l_nan:
                gg.loc[k, 'date_time_init'] = gg.loc[k-1, 'date_time_end']
                gg.loc[k, 'date_time_end'] = datetime.strptime(str(gg.loc[k, 'date_time_init']), "%Y-%m-%d %H:%M:%S") + timedelta(seconds=gg.loc[k, 'time_travel'])
        i += 1

print(list_samples[0].to_string())

In [ ]:
# df_sp.to_csv('df_sp.csv')
df_sp = pd.read_csv('df_sp.csv')
df_sp.mask((df_sp < 0) | (df_sp > 305), np.nan, inplace=True)
df_sp.drop(columns=df_sp.columns[0], inplace=True)
df_sp

In [ ]:
print(f"Rutas admitidas: {df_his[df_his['ida_Points'] > 99].shape[0]}")
print("Rutas no admitidas: ")
empty_stops = []
for i, row in df_his.iterrows(): 
    if row['ida_Points'] <= 99: 
        print(f" veh: {row['vehicle']}, lap: {row['lap']}, nans: {row['ida_Points']}")
        empty_stops += row['stops_nan']

print(Counter(empty_stops))

In [ ]:
import matplotlib.pyplot as plt

# lista = [1, 2, 3, 1, 2, 3, 3, 3, 4, 5]
lista = list(df_sp['link-0'].dropna())
repeticiones = {}

for n in lista:
    if n in repeticiones:
        repeticiones[n] += 1
    else:
        repeticiones[n] = 1

x = list(repeticiones.keys())
y = list(repeticiones.values())

plt.bar(x, y)
plt.show()

df_sp['link-0'].dropna().mean()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Generar datos de ejemplo
data = df_sp['link-0'].dropna().to_numpy()

# Ajustar una distribución normal a los datos
mu, std = norm.fit(data)

# Crear un histograma de los datos
plt.hist(data, bins=20, density=True, alpha=0.6, color='g')

# Crear una curva de la distribución normal ajustada
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

# Añadir etiqueta
# Añadir etiquetas y título
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.title('Distribución normal de los datos')

# Mostrar la gráfica
plt.show()